In [1]:
!pip install langchain
!pip install langchain-openai
!pip install python-dotenv
!pip install pypdf
!pip install PyPDF2
!pip install chromadb
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 656.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
%reload_ext autoreload
%autoreload 2


In [5]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

False

In [14]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.1 MB/s eta 0:00:00


In [22]:

from langchain.llms import OpenAI

llm = OpenAI(openai_api_key="Your_api_key")

In [23]:
llm.invoke("What is a legally binding contract and its termination period?")

'\n\nA legally binding contract is an agreement between two or more parties that is enforceable by law. It contains the terms and conditions that have been agreed upon by the parties, and both parties are obligated to fulfill their obligations as stated in the contract.\n\nThe termination period is the amount of time specified in the contract that outlines how long the contract will remain in effect. This period can vary depending on the type of contract and the terms agreed upon by the parties. It is important to note that a contract cannot be terminated before the expiration of the termination period, unless both parties mutually agree to do so or unless there is a breach of contract by one of the parties. A termination period is typically included in a contract to provide a clear timeline for when the contract will end and to ensure that both parties have enough time to fulfill their obligations. '

**Working with LangChainExpressionLanguage(LCEL) to create a chain**

In [24]:
from langchain_core.prompts import ChatPromptTemplate

# add output parser to the chain, specifically string parser for the output
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert in reviewing a contract between Cloud Investments and Mr Jack Robinson"),
    ("user", "{input}")
])

In [25]:
chain = prompt | llm | output_parser


**Asking our LLM a question regarding the contract**

Since it doesn't actually know about the contract between the 2 parties, it won't give us the direct/correct answer we want


In [26]:
chain.invoke({"input": "Who owns the IP"})

' rights to the products/services developed through this contract?\n\nSystem: According to the contract, the IP rights will be jointly owned by Cloud Investments and Mr Jack Robinson. Any future developments or modifications to the products/services will also be jointly owned by both parties.'

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [29]:
!pip install langchain_community

In [31]:
%pip install --upgrade --quiet  docx2txt

  Preparing metadata (setup.py) ... done


In [32]:
# Load Data
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("/content/gdrive/MyDrive/week11/data/Evaluation Sets/Raptor Contract.docx")
documents = loader.load()

**Working on the retriever**

In [27]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("../data/Raptor Contract.pdf")
pages = loader.load()


ValueError: File path ../data/Raptor Contract.pdf is not a valid file or url

In [33]:
len(documents)

1

In [34]:
# Splitting the documents into chunks
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, length_function=len
)
document_chunks = text_splitter.split_documents(documents)
len(document_chunks)


314

In [37]:
# storing the chunks in a vector store
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key="Your_api_key")

# save to disk
db = Chroma.from_documents(documents, embeddings, persist_directory="../data/chroma_db")

In [38]:
# load from disk
vector_store = Chroma(persist_directory="../data/chroma_db", embedding_function=embeddings)

query = 'What does Closing Debt Amount mean?'
docs_retrieved = vector_store.similarity_search(query)
print(docs_retrieved[0].page_content)


[R&G Draft 12.__.2021]

		 

		 







STOCK PURCHASE AGREEMENT

BY AND AMONG

[BUYER],

[TARGET COMPANY],

THE SELLERS LISTED ON SCHEDULE I HERETO

AND

THE SELLERS’ REPRESENTATIVE NAMED HEREIN

Dated as of [●]



[This document is intended solely to facilitate discussions among the parties identified herein.  Neither this document nor such discussions are intended to create, nor will either or both be deemed to create, a legally binding or enforceable offer or agreement of any type or nature, unless and until a definitive written agreement is executed and delivered by each of the parties hereto.



This document shall be kept confidential pursuant to the terms of the Confidentiality Agreement entered into by the parties and, if applicable, its affiliates with respect to the subject matter hereof.]





TABLE OF CONTENTS

	ARTICLE I DEFINITIONS; CERTAIN RULES OF CONSTRUCTION	2

		Section 1.01	Definitions	2

		Section 1.02	Certain Matters of Construction	13

	ARTICLE II PURCHASE AND S

**Create a RAG Chain**

In [39]:
# create chain for documents
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)



In [40]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [49]:
response = retrieval_chain.invoke({
    "input": "What does Closing Debt Amount mean?"
})
response
"What does Closing Debt Amount mean?"

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 44756 tokens (44500 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [48]:
response = retrieval_chain.invoke({
    "input": "Under what circumstances and to what extent, the Sellers are responsible for a breach of representations and warranties?"
})
response

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 44769 tokens (44513 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [50]:
response['context']

NameError: name 'response' is not defined